# 2023年のチームスタイルの指標を取得する
[FootballLab](https://www.football-lab.jp/summary/team_style/j1/?year=2023)

In [14]:
import pandas as pd
import numpy as np
from collections import OrderedDict

import re
import requests
from bs4 import BeautifulSoup
import time

In [15]:
#urlを変数に格納
url = "https://www.football-lab.jp/summary/team_style/j1/?year=2023"

#HTMLを取得して変数に格納
match_html = requests.get(url).text

# 取得したHTMLデータを変数に格納
## html.parser：htmlタグの解析と、データの取得ができる
soup = BeautifulSoup(match_html, 'html.parser')

In [27]:
# 指数一覧表から、（href属性で"s="を持つ箇所を対象に18*8(144)セル分だけデータを取り出す
data_param = soup.find_all(href=re.compile("s="))

# 長さ144のリストを用意
param_list = [0]*144
for i in range(144):
    # 順にテキスト部分のみ格納する
    param_list[i] = int(data_param[i].text)
#18*8に成型したものをデータフレームにする。チーム名と結合するため、インデックスを作成する
df_params = pd.DataFrame(np.reshape(param_list, (18,8)),
                         columns=[
                         'atk.set.play',
                         'left.side.attack',
                         'center.attack',
                         'right.side.attack',
                         'short.counter',
                         'long.counter',
                         'enemy.camp.possession',
                         'self.camp.possession'
                         ]).reset_index()

In [4]:
# aタグを用いられたhtmlをリスト形式ですべて取得
data_teams = soup.find_all("a")
# ベースURLを格納する
teams_url_dom = data_teams[0].get('href')
# '/'で始まり、'/'で終わるhref属性のみ(パスコンポーネント)をリストに格納
# 先頭の条件部分は、href属性が存在しないaタグがあってもエラーが発生しないようにする対応
teams_path_list = \
    [team.get('href') for team in data_teams \
        if team.get('href') \
            and team.get('href').startswith('/') \
            and team.get('href').endswith('/')]
    
# 必要な部分のみスライス、重複の除去
# インデックスを基準にパラメータのデータフレームと結合させる必要がある
# 順序は変えたくないよう、forの外側で処理
teams_path_list = teams_path_list[81:115]
teams_path_list = list(OrderedDict.fromkeys(teams_path_list))
# チーム名を追加するため、空のリストを用意
team_list = []
# 取得したしたベースurlとチームごとのパスを用いて、
# チームごとのページから英名でチームを取得する
for i in range(len(teams_path_list)):
    # チームごとのurlを生成する
    team_url = teams_url_dom + teams_path_list[i]
    
    #チームページからHTMLを取得する
    match_html = requests.get(team_url).text

    # 取得したHTMLデータを変数に格納
    soup = BeautifulSoup(match_html, 'html.parser')
    
    # 指数一覧表から、（href属性で"s="を持つ箇所を対象に18*8(144)セル分だけデータを取り出す
    data_en = soup.find_all('span', class_='eng')
    team_list = np.append(team_list, data_en[1].text)
    time.sleep(2)

# チーム名を順番にデータフレームに変換する
df_team = pd.DataFrame(team_list, columns=['team.name']).reset_index()

In [28]:
# データフレームに結合
df_team_style = pd.merge(df_team, df_params, on='index').drop(columns={'index'})

In [29]:
df_team_style.to_csv('team_style_2023.csv', index=False)